In [167]:
import pickle
import tensorflow as tf
import random
import numpy as np

In [169]:
#load testing data and tokenizer
with open('./testing_data3.pkl', 'rb') as f:
    data = pickle.load(f)
with open('./tokenizer3.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


In [171]:
#Load Model
model = tf.keras.models.load_model('./model_v3_final',  compile=False)

In [208]:
#Function for getting the data. IT was build for an old desing, and therefore has extra fucntionalily
#But for getting a random starting poitn, it works fine
def data_iteration(batch_size_per_author):
    starts = [random.randint(0,len(data[0])-batch_size_per_author-1), random.randint(0,len(data[1])-batch_size_per_author-1), random.randint(0,len(data[2])-batch_size_per_author-1), random.randint(0,len(data[3])-batch_size_per_author-1), random.randint(0,len(data[4])-batch_size_per_author-1),random.randint(0,len(data[5])-batch_size_per_author-1)]
    lis_total = data[0][starts[0]:starts[0]+batch_size_per_author] + data[1][starts[1]:starts[1]+batch_size_per_author] + data[2][starts[2]:starts[2]+batch_size_per_author] + data[3][starts[3]:starts[3]+batch_size_per_author] + data[4][starts[4]:starts[4]+batch_size_per_author] + data[5][starts[5]:starts[5]+batch_size_per_author]
    X = np.asarray([x[0] for x in lis_total])
    y_class = np.asarray([x[1] for x in lis_total])
    y_gen = tf.keras.utils.to_categorical([x[2] for x in lis_total], num_classes =(len(tokenizer.word_index) + 1))
    return X, y_class, y_gen

In [209]:
batch_size_per_author = 1
totalX, total_class_y, total_gen_y = data_iteration(batch_size_per_author)

In [210]:
print(len(data[4]))

70194


In [211]:
from tensorflow import keras
from tensorflow.keras import layers
#Set batch size, necessary for stateful RNN

#This is the definition of all the shared layers. B/C of stateful RNN and the classifier/generator split, these will be reused in all the models
embedding = layers.Embedding(len(tokenizer.word_index) + 1, 750, input_length=100)
RNN1 = layers.GRU(2250, stateful=True)
dense1 = layers.Dense(1250)

#These are generator/classifier specific layers. Generator layer is shared by all diff generators. Classifier is not, but it's here for organizational consistency
generator_dense_final = layers.Dense(len(tokenizer.word_index) + 1, activation='softmax')
classifier_dense_final = layers.Dense(6, activation='softmax')
#The model path. Different outputs for gen and class, but uses the same structure mostly
inputs = keras.Input(batch_input_shape=(1,100))
x = embedding(inputs)
x = RNN1(x)
# x = RNN2(x)
# x = RNN3(x)
x = dense1(x)
generator_output = generator_dense_final(x)
classifier_output = classifier_dense_final(x)
model2 = keras.Model(
    inputs=[inputs],
    outputs=[generator_output, classifier_output],
)
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [212]:
old_weights = model.get_weights()
model2.set_weights(old_weights)

In [228]:
indiv = totalX[1]
print(indiv)
indiv = indiv.reshape(100,1).transpose()
print(indiv.shape)
result = model2.predict(indiv)
print(result[0])
print(result[0].shape[1])
print(sum(result[0].tolist()[0]))

[  426     2    59    42    60 13975     5     3   174     5   297     3
    77   224     1    70   293   481    61     1    21     3  2345    11
    28    16 23582  4832     2  6379 10354     2    12    26    63 16716
   163     1     4   528    50  2991  1036   466     1    23    50  5898
 40398    81     7  2049   383     1    11  4618  5162   199     1   168
 23583  6145     1     4  6269     1    64     3  5564     1 17389 40399
   125     2  9085     2     3   869    11     3   265    35   216   303
    24    29   113  1017    18    74   318    90   785   819    21   223
    63 12423   482     2]
(1, 100)
[[2.9547450e-14 1.8004086e-05 6.0631868e-07 ... 2.5121595e-14
  3.2755554e-14 2.6182685e-14]]
55396
1.0000014830942565


In [229]:
import sklearn
predicted_id = random.choices(range(55396), weights = result[0][0])

In [230]:
print(predicted_id)

[30]


In [234]:
indiv = totalX[1]
indiv = indiv.reshape(100,1).transpose()
len_to_predict = 100
for i in range(len_to_predict):
    print(indiv[:,-100:].shape)
    result = model2.predict(indiv[:,-100:])[0].reshape(55396)

    predicted_id = random.choices(range(result.shape[0]), weights = result)
    predicted_id = np.asarray(predicted_id).reshape(1,1)
    indiv = np.concatenate((indiv, predicted_id), axis = 1)

(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)
(1, 100)


In [235]:
print(np.ndarray.tolist(indiv)[0])

[426, 2, 59, 42, 60, 13975, 5, 3, 174, 5, 297, 3, 77, 224, 1, 70, 293, 481, 61, 1, 21, 3, 2345, 11, 28, 16, 23582, 4832, 2, 6379, 10354, 2, 12, 26, 63, 16716, 163, 1, 4, 528, 50, 2991, 1036, 466, 1, 23, 50, 5898, 40398, 81, 7, 2049, 383, 1, 11, 4618, 5162, 199, 1, 168, 23583, 6145, 1, 4, 6269, 1, 64, 3, 5564, 1, 17389, 40399, 125, 2, 9085, 2, 3, 869, 11, 3, 265, 35, 216, 303, 24, 29, 113, 1017, 18, 74, 318, 90, 785, 819, 21, 223, 63, 12423, 482, 2, 903, 1, 8, 55, 24, 123, 1, 103, 7, 76, 581, 582, 3, 1552, 262, 6, 3, 248, 2, 21, 167, 7910, 13, 86, 28, 5, 199, 4, 135, 28, 3, 90, 129, 2, 4, 58, 11, 4721, 81, 12, 60, 13, 361, 22, 4870, 12091, 2, 167, 755, 621, 42, 52, 120, 47, 20, 33, 1, 4, 8, 104, 66, 3, 114, 9, 3, 491, 2715, 2, 30, 58, 278, 159, 5, 1066, 2, 4, 39, 8, 88, 1, 720, 3300, 7, 56, 54, 108, 179, 12, 100, 19, 361, 1, 50, 6, 61, 33, 839, 9, 361, 3]


In [236]:
a = tokenizer.sequences_to_texts(np.ndarray.tolist(indiv))

In [237]:
print(a)

['please . then they will hasten to the door to call the little child , who cannot thank them , for the ice that on her lisping piled . x immortality . it is an honorable thought , and makes one lift ones hat , as one encountered gentlefolk upon a daily street , that weve immortal place , though pyramids decay , and kingdoms , like the orchard , flit russetly away . xi . the distance that the dead have gone does not at first appear ; their coming back seems possible for many an ardent year . besides , i do not think , its a good sense beyond the playing part of the world . for long hating you go on to place and look on the back way . and now that depend upon it will you turn ? rupa snarled . long months ago they were even when she said , and i dont know the house in the country club . but now im going to seek . and if i am , id tread a up or two through it before my turn , one of them said quietly in turn the']
